## Model Training

### Spark Session Setup

In [1]:
import findspark

findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('modelTraining').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/22 12:07:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
#data loading will change once data is hosted on hadoop, loading from local for now
from pyspark import SparkFiles
df = spark.read.csv('file:///home/jj/cs354/final_project/cs354-fraud-checker/credit_card_transactions.csv', inferSchema = True, header = True) #hardcoded filepath for Brian's system for now, will be changed once data is on hadoop
#also noted, this is temp with raw data, but will be loading clean data

### Train Test Split

In [8]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed = 101)

### Model Importing and Setup

In [10]:
#import models
from pyspark.ml.classification import NaiveBayes

#set up models with default settings
nb = NaiveBayes()

In [11]:
#eval set up
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator()

### Hyperparameter Tuning

In [13]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#naive bayes
grid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]).build()
nb_cv = CrossValidator(estimator = nb, estimatorParamMaps = grid, evaluator = acc_evaluator, numFolds = 5, parallelism = 4)

In [ ]:
#run predictions
nb_cv_model = nb_cv.fit(train_data)
nb_cv_pred = nb_cv_model.transform(test_data)

In [ ]:
#results
print(cv_model.bestModel)

nb_accuracy = evaluator.evaluate(nb_cv_pred)
print(f"Naive Bayes Accuracy: {nb_accuracy}")